# **NAPAD NA RSA**

**Avtor**: Nina Mislej
<br />**Vpisna številka**: 63200016

### **Kratek opis naloge**
Imamo pametne kartice in vsaka izmed teh ima svoj **ključ** za šifriranje z **DES** algoritmom. Skrivno število uporabnika **A** označimo z **u**(**A**). Ker je sistem zastarel bo implementiran nov sistem, ki za šifriranje uporablja **ista števila** vendar je algoritem za šifriranje po novem **RSA**. Imamo strežnik, kjer se preverjajo podatki kartice. Ključa sta označena kot (**n,e**) in (**n,d**), kjer je (**n,e**) javni ključ uporabljen za šifriranje in (**n,d**) privatni ključ za dešifriranje. Zato, da je šifriranje hitro uporabimo ključ velikosti **e** = **3**. Ta ključ doda na vsako kartico.

Ker so sporočila z tako majhnim eksponentom tvegana, je uporabljena varna shema z različnim **zapolnjevanjem**. Vsaka prijava ima torej različno število za zapolnjevanje.

## **PRIMER A**

Omogočeno je **spremljanje** poslanih sporočil na pametni kartici in strežniku. Število, ki ga šifriramo je generirano tako: **m<sub>i</sub>** = **u**(**A**) · **2<sup>123</sup>** + **i**, kjer je i zaporedna številka sporočila. 
<br/> Za ta specifičen primer želimo izračunati:
- **u**(**Bojan**) =  **34100010166843172** 
- zaporedna številka **i** = **1909**.
- ključ: (**n**, **e**) = (**1353040922319896710729948440742113526140662069124237571**, **3**)

### **Rešitev**
RSA algoritem uporablja za šifriranje in dešifriranje sledeč sistem. Vzamemo števili **p** in **q**, ki jih javnost ne ve in jih zmnožimo v število **n**, ki je nam in javnosti kasneje znano. 
</br>Nato vzamemo **Eulerjevo funkcijo**, ki bo zmnožek števil **(p - 1)** in **(q - 1)**. Tudi to število ostane skrito.
</br>Vzamemo število e, ki je tuje φ(n).
</br>Ključ za dešifriranje generiramo tako: **d** ≡ **e<sup>−1</sup>** mod φ(n)

**ŠIFRIRANJE:** **c** ≡ **m<sup>e</sup>** mod n
</br>**DEŠIFRIRANJE:** **m** = **c<sup>d</sup>** mod n

In [1]:
def get_m(u, i):
    m = u * pow(2,123) + i
    return m

In [2]:
def encrypt(m, e, n):
    c = pow(m, e, n)
    return c

In [3]:
i = 1909
u = 34100010166843172
e = 3
n = 1353040922319896710729948440742113526140662069124237571

m = get_m(u, i)
c = encrypt(m, e, n)
print(f'To je skrivno stevilo z zaporedno stevilko {i}:\n{m}\n')
print(f'Zasifrirano stevilo pa je:\n{c}')

To je skrivno stevilo z zaporedno stevilko 1909:
362613505362545633945092350256402989265709266502682485

Zasifrirano stevilo pa je:
723941917581347359534709894322925645398915405217429002


## **PRIMER B**

**Cene** je zasledil **3 Anitine** zaporedne prijave za dostop do računalnika.
- **c<sub>j</sub>** = **164867525413631686108542244605590332657131844994186648**
- **c<sub>j+1</sub>** = **669330273667331356154438891368274712878935740757554990**
- **c<sub>j+2</sub>** = **853109242122207805055956523445529343243869885591747755**

Javni ključ in shema za zapolnjevanje sta enaka. Zaporednega števila ne poznamo vendar je veliko manjše od **2<sup>123</sup>** in bo zapisano v zgornjih 56 bitih skrivne številke **m<sub>j</sub>** dešifriranjega kriptograma **c<sub>j</sub>**. 
</br> Dešifrirati želimo Anitino skrivno število.

### **Rešitev**

Števila **m<sub>j</sub>**, **m<sub>j+1</sub>** in **m<sub>j+2</sub>** so zaporedna števila, torej je:
- **m<sub>j</sub>** = **u** · **2<sup>123</sup>** + **j**
- **m<sub>j+1</sub>** = **m<sub>j</sub>** + **1** = **u** · **2<sup>123</sup>** + **j** + **1**
- **m<sub>j+2</sub>** = **m<sub>j</sub>** + **2** = **u** · **2<sup>123</sup>** + **j** + **2**

Torej je razlika med števili zelo majhna in lahko uporabimo nekaj idej **Franklin–Reiterjevega izreka**. Predpostavka je tudi, da imamo več prejetih sporočil in majhen eksponent, kar v našrem primeru drži saj je eksponent 3 in imamo 3 zaporedna prejeta sporočila. 

Definirajmo 3 polinome:
- **p<sub>j</sub>**(**x**) = **x<sup>e</sup>** - **c<sub>j</sub>** mod n
- **p<sub>j+1</sub>**(**x**) = (**x** + **1**)**<sup>e</sup>** - **c<sub>j+1</sub>** mod n
- **p<sub>j+2</sub>**(**x**) = (**x** + **2**)**<sup>e</sup>** - **c<sub>j+2</sub>** mod n

Ti polinomi imajo skupno ničlo **m<sub>j</sub>**, saj velja:
- **c<sub>j</sub>** = **m<sub>j</sub><sup>e</sup>** mod n
- **c<sub>j+1</sub>** = **m<sub>j+1</sub><sup>e</sup>** mod n = (**m<sub>j</sub>** + **1**)**<sup>e</sup>** mod n
- **c<sub>j+2</sub>** = **m<sub>j+2</sub><sup>e</sup>** mod n = (**m<sub>j</sub>** + **2**)**<sup>e</sup>** mod n

Ker imajo skupno ničlo lahko poskusimo izračunati **GCD**. Če dobimo linearni člen bomo našli ničlo **m<sub>j</sub>** in potem nadaljevali z obračanjem sheme za zaponlnjevanje. Najprej poskusimo z samo dvemi sporočili. Če bomo potrebovali še eno bomo zračunali GCD vseh treh.

In [4]:
c0 = 164867525413631686108542244605590332657131844994186648
c1 = 669330273667331356154438891368274712878935740757554990
c2 = 853109242122207805055956523445529343243869885591747755

In [5]:
from sympy.abc import x
import sympy as sy

p0 = sy.Poly(x**e - c0, modulus=n)
p1 = sy.Poly((x + 1)**e - c1, modulus=n)
p2 = sy.Poly((x + 2)**e - c2, modulus=n)

root = sy.gcd(p0,p1)
print(f"To je najvecji skupni delitelj:\n{root.as_expr()}\n")

m = - root(0)
print(f"To je prvo zakodirano skrito stevilo:\n{m}")



To je najvecji skupni delitelj:
x - 172059523753512248264261571009447296047298719699176998

To je prvo zakodirano skrito stevilo:
172059523753512248264261571009447296047298719699176998


Sedaj imamo **m**. Naša naslednja naloga je, da ven dobimo skrivno število.

In [6]:
def num_to_bin(n):
    return [int(bit) for bit in list(bin(n))[2:]]

def bin_to_num(seq):
    s = "0b" + "".join(str(bit) for bit in seq)
    return int(s, 2)

In [7]:
bin_m = num_to_bin(m)
len_m = len(bin_m)

print(f"Prej pridobljen m ima {len_m} bitov.")

Prej pridobljen m ima 177 bitov.


Če si bolj podrobno ogledamo funkcijo, ki je število spremenila opazimo, da lahko množenje z **2<sup>123</sup>** interpretiramo kot zamik za 123 bitov. Na začetku naloge je opisano, da so bila skrita števila na karticah najprej ključi za **DES** algoritem. Za algoritem je značilno, da ima **ključe dolžine 64 bitov** od tega jih je samo 56 na voljo za izbiro števila. Število je torej zapisano na največ **56 bitih** in premaknjeno za 123 bitov. Prav tako vemo, da je zaporedno število manjše od **2<sup>123</sup>** in ga prištejemo šele po premiku, torej se števili ne prekrivata. Izrecno piše, da zasede samo prvih 57 bitov, kar nam zadevo še olajša. Odštejemo zaporedno število in u zamaknemo nazaj v desno z deljenjem. 

Dobimo končni rezultat: **16180399854194147**

In [8]:
seq_num = bin_to_num(bin_m[121:])
u = (m - seq_num) / pow(2,123)

print(f"Končen rezultat: {u}")
print(f"Zaporedna številka: {seq_num}")

Končen rezultat: 16180399854194147
Zaporedna številka: 3622


Zadevo lahko seveda še preverimo:

In [9]:
test_m = get_m(u, seq_num)
test_c = encrypt(test_m, e, n)

print(f"Ali sta šifrirani števili enaki: {c0 == test_c}")

Ali sta šifrirani števili enaki: True


## **PRIMER C**

Shema iz prejšnjih nalog je bila za šifriranje preveč zapletena, zato posodobijo protokol za izdajo skrivnih števil. Izbrali so **30**-**bitna** števila za vsakega uporabnika. Strežnik kartici pošlje **80**-**bitni** izziv **s** in kartici dovoli dostop če mu odgovori s šifriranjem besedila **m** = **u**(**A**) + **2<sup>99</sup>** · **s**. Javni ključ je isti kot prej. 

Prestreženo je bilo šifrirano število **c** = **1304427715737794183639527703843118636234043562220564999**.
</br>Vemo tudi da je bil poslan **s** = **661795599945365472793374** 
</br>Izvesti želimo napad na skrivno število **u** z zgoraj navedenimi podatki!


In [10]:
c = 1304427715737794183639527703843118636234043562220564999
s = 661795599945365472793374

In [11]:
import math

def get_m(s, u):
    m = (pow(2,99) * s) + u 
    return m

### **Rešitev**

**REŠITEV**: 1003453592

Lahko si pomagamo tako kot prej, vendar za število **m** dejansko vzamemo **s** in gledamo **u** kot zapolnitev. Morda si lahko pomagamo s **Coppersmithovim napadom** saj je padding, oziroma naše iskano število, kratko - samo 30 bitov. Ker vemo **shemo**, lahko generiramo še eno število recimo **u** = **0** in dobimo:
- **m <sub>generiran</sub>** = **2<sup>99</sup>** · **s** + **0**
- **m <sub>original</sub>** = **2<sup>99</sup>** · **s** + **u**

Novo generirano število nato še šifriramo v **c <sub>generiran</sub>**.
</br>To lahko uporabimo, saj sta števili **u** in **0** obe **manjši** od **2<sup>99</sup>**, ker je **u** največ velikosti **2<sup>31</sup>** - **1**

In [12]:
c_generiran = encrypt(get_m(s,0), e, n)
c_original = c

print(f'Naše zakriptirano število:\n{c_generiran}\n')
print(f'Originalno zakriptirano število:\n{c_original}')

Naše zakriptirano število:
250709091204979875045912708275270346215666699961305924

Originalno zakriptirano število:
1304427715737794183639527703843118636234043562220564999


Sedaj lahko uporabimo **Coppersmithov izrek**, da najdemo razliko med **0** in **u**, ki očitno kar **u**.

</br>Tako kot prej, lahko vidimo, da velja: **m <sub>original</sub>** = **m <sub>generiran</sub>** + **u**

Spet definiramo 2 para po 2 polinoma:
- **g <sub>generiran</sub>** (**x**, **u**) = **x<sup>e</sup>** - **c <sub>generiran</sub>** mod n
- **g <sub>original</sub>** (**x**, **u**) = (**x** + **u**)**<sup>e</sup>** - **c <sub>original</sub>** mod n

Sedaj moramo najti rezultanto teh dveh polinomov: **h**(**y**) = **res<sub>x</sub>** ( **g <sub>generiran</sub>**, **g <sub>original</sub>** ) ∈ **Z<sub>n</sub>** in nato njene ničle. 
</br>Ničle bomo iskali po **Coppersmithovi metodi**.

In [13]:
from sympy.abc import x, u
import sympy as sy
sy.init_printing() 

g_generiran = sy.Poly(x**e - c_generiran, modulus=n)
g_original = sy.Poly((x + u)**e - c_original, modulus=n)

rez = sy.resultant(g_generiran, g_original, modulus=n)
print(rez.as_expr())

u**9 - 455074028958649504320948105219317817773806448529302083*u**6 - 464059992402711991548196570902245393779212857554194246*u**3 - 422347353194341057197606878350612784897058912804841381


Sedaj moramo preveriti ali ustreza pogojem za to metodo. Veljati mora, da je iskana **ničla po absolutni vrednosti manjša** od **n** na $\frac{1}{deg(p)}$, kjer je **deg**(**p**) stopnja polinoma. 
</br>Funkcija deluje tako, da vrne **False** če ima pogojno število manj bitov kakor potencialno največja ničla, ki ima lahko 30 bitov. Drugače vrne **True**

In [14]:
def check_coppersmith(deg, n):
    cond = math.ceil(n**(1/deg))
    no_bits = len(num_to_bin(cond))
    if no_bits <= 30: return False
    else: return True 

print(f'Ali polinom ustreza pogojem za metodo: {check_coppersmith(sy.degree(rez),n)}')

Ali polinom ustreza pogojem za metodo: False


Torej iskanje po tej metodi za ta polinom najverjetneje ne bo uspelo. Vseeno lahko poskusimo.
</br> pomagamo si z programsko opremo za simbolno matematiko, ki ima ta algoritem že implementiran: **SageMath**

In [1]:
# Has to be executed either afterwards or separately using MathSage
K = Zmod(1353040922319896710729948440742113526140662069124237571)
P.<u> = PolynomialRing(K)
rez = u**9 - 455074028958649504320948105219317817773806448529302083*u**6\
           - 464059992402711991548196570902245393779212857554194246*u**3\
           - 422347353194341057197606878350612784897058912804841381
        
print(rez.small_roots())

[]


Vidimo, da vrne prazen seznam, torej ničla ni bila najdena. Naslednji korak je, da dobimo **polinom nižje stopnje**, s skupno ničlo. 

Vzeli bomo še eno število npr. **1** in izračunali njegovo šifrirano vrednost. Dobimo: **m<sub> pomozen</sub>** = **2<sup>99</sup>** · **s** + **1**
</br> Sedaj spet čisto ekivalentno ponovimo prejšnje korake, velja:
- imamo zvezo: **m<sub> original</sub>** = **m<sub> pomozen</sub>** + **u** - **1**
- definiramo 2 nova polinoma: 
     - **h<sub> generiran</sub>** (**x**, **u**) = **x<sup>e</sup>** - **c<sub> pomozen</sub>** mod n
     - **h<sub> original</sub>** (**x**, **u**) = (**x** + **u** - 1)**<sup>e</sup>** - **c<sub> original</sub>** mod n

In [16]:
c_pomozen = encrypt(get_m(s,1), e, n)

h_generiran = sy.Poly(x**e - c_pomozen, modulus=n)
h_original = sy.Poly((x + u - 1)**e - c_original, modulus=n)

rez_2 = sy.resultant(h_generiran, h_original, modulus=n)

root = sy.gcd(rez, rez_2)
print(root.as_expr())

u**3 - 94652538071181126988218514488049207738801189623269635*u**2 + 350860114408333717151340742022780238302215777475914718*u + 299322297787082402136333445174265236122285206864978496


Razložimo. Polinoma imata za skupno ničlo število **u**, ki ga iščemo. Če ne bi namesto **h<sub> original</sub>** vzeli kar prejšnji **g<sub> original</sub>** bi bila ničla rezultante **u** - **1**, saj bi to bila razlika med prištetima elementoma k **2<sup>99</sup>** · **s**. Ker pa smo v novem polinomu dodali **-1**, bo morala ničla biti **za 1 večja** torej **u**. To lahko pokažemo tako, da v polinom vstavimo **u** + **1**. 

**h<sub> original</sub>** (**x**, **u** + **1**) = (**x** + (**u** + **1**) - 1)**<sup>e</sup>** - **c<sub> original</sub>** mod n = **g<sub> original</sub>**.

Polinoma imata potemtakem očitno skupno ničlo **u** in ko smo izračunali **GCD** se je stopnja polinoma zmanjšala. Spet poskusimo ali ustreza pogojem. 

In [17]:
print(f'Ali polinom ustreza pogojem za metodo: {check_coppersmith(sy.degree(root),n)}')

Ali polinom ustreza pogojem za metodo: True


In [2]:
# Has to be executed either afterwards or separately using MathSage
root = u**3 - 94652538071181126988218514488049207738801189623269635*u**2\
            + 350860114408333717151340742022780238302215777475914718*u\
            + 299322297787082402136333445174265236122285206864978496
roots = root.small_roots()
print(roots)

[1003453592]


MathSage je tokrat našel ničlo. Preverimo ali je to res pravilen ogovor ... 

**REŠITEV:** 1003453592  

In [18]:
u = 1003453592
m_test = get_m(s, u)
c_test = encrypt(m_test, e, n)

print(f"Ali sta šifrirani števili enaki: {c0 == test_c}")

Ali sta šifrirani števili enaki: True
